In [26]:

import numpy as np
import glob
import mccd

%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [27]:

data_dir_path = '/n05data/tliaudat/new_deepmccd/training_realistic_sims/output_mccd/trained_models/'
model_pattern = 'fitted_model-*.npy'
save_dir_path = '/n05data/tliaudat/new_deepmccd/training_realistic_sims/output_mccd/eigenPSF_datasets/'

file_paths = glob.glob(data_dir_path + model_pattern)

short_file_paths = [file_paths[0], file_paths[1]]


In [28]:
short_file_paths

['/n05data/tliaudat/new_deepmccd/training_realistic_sims/output_mccd/trained_models/fitted_model-2200000.npy',
 '/n05data/tliaudat/new_deepmccd/training_realistic_sims/output_mccd/trained_models/fitted_model-2200001.npy']

In [29]:

global_eigenPSFs = []
local_eigenPSFs = []

for path in short_file_paths:
    
    fitted_model = np.load(path, allow_pickle=True)
    S = fitted_model[1]['S']
    
    global_eigenPSFs.append(mccd.utils.reg_format(S[-1]))

    for k in range (40):
        local_eigenPSFs.append(mccd.utils.reg_format(S[k]))

# Concatenate list to np.ndarray
global_eigenPSFs = np.concatenate(global_eigenPSFs, axis=0)
local_eigenPSFs = np.concatenate(local_eigenPSFs, axis=0)

# Shuffle vignets
np.random.shuffle(global_eigenPSFs)
np.random.shuffle(local_eigenPSFs)



In [30]:

save_dic = {'VIGNETS_NOISELESS': global_eigenPSFs}
mccd.mccd_utils.save_to_fits(save_dic, save_dir_path + 'test_global_eigenpsfs.fits')

save_dic = {'VIGNETS_NOISELESS': local_eigenPSFs}
mccd.mccd_utils.save_to_fits(save_dic, save_dir_path + 'test_local_eigenpsfs.fits')

save_dic = {'VIGNETS_NOISELESS': np.concatenate((global_eigenPSFs, local_eigenPSFs), axis=0)}
mccd.mccd_utils.save_to_fits(save_dic, save_dir_path + 'test_all_eigenpsfs.fits')


In [31]:
from astropy.io import fits

In [32]:

saved_data = fits.open(save_dir_path + 'test_all_eigenpsfs.fits')


In [34]:
saved_data[1].data['VIGNETS_NOISELESS'].shape

(1210, 51, 51)

In [47]:

imshow(saved_data[1].data['VIGNETS_NOISELESS'][220,:,:]);colorbar()
